In [1]:
model_path = "./models/vosk-model-ru-0.10"

In [2]:
import json, os, sys, wave
import matplotlib.pyplot as plt
from vosk import Model, KaldiRecognizer
import pyaudio

In [8]:
def record(player, seconds=7):
    leng = 22050 * seconds // 1024
    stream = player.open(input=True, format=pyaudio.paInt16, channels=1, rate=22050, 
                         input_device_index=2,
                         frames_per_buffer=leng)
    data = stream.read(leng)
    stream.stop_stream()
    stream.close()
    return data

In [4]:
def recognize(model, frames):
    rec = KaldiRecognizer(model, 22050)
    words = []
    if rec.AcceptWaveform(frames): words = rec.Result()
    else: words = rec.PartialResult()
    jsons = [json.loads(w) for w in words]
    texts = [w['text'] for w in jsons if 'text' in w and w['text']]
    text = "\n".join(texts)
    return text

In [ ]:
pya = pyaudio.PyAudio()
frames = record(pya, 7)
plt.plot(frames)
plt.show()

In [ ]:
model = Model(model_path)
recognize(model, frames)